# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [15]:
# Import necessary packages
import sqlite3
import pandas as pd

In [16]:
# Create the database school.sqlite 
con = sqlite3.connect('school.sqlite')
cur = con.cursor()

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [17]:
# Your code here
cur.execute(
"""
CREATE TABLE contactInfo(
                      id INTEGER PRIMARY KEY,
                      first_name TEXT,
                      last_name TEXT,
                      role TEXT, 
                      telephone_number INTEGER,
                      street TEXT,
                      city TEXT,
                      state TEXT,
                      zipcode INTEGER
)
"""
)



OperationalError: table contactInfo already exists

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [24]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)
contacts[0].keys()

dict_keys(['firstName', 'lastName', 'role', 'telephone ', 'street', 'city', 'state', 'zipcode '])

In [27]:
# Iterate over the contact list and populate the contactInfo table here
for contact in contacts:
  cur.execute(
    """
  INSERT INTO contactInfo(first_name, last_name, role, telephone_number, street, city, state, zipcode)
                  VALUES(?,?,?,?,?,?,?,?);
  """, (contact['firstName'], contact['lastName'], contact['role'], contact['telephone '], contact['street'], contact['city'], contact['state'], contact['zipcode '])
  )

**Query the Table to Ensure it is populated**

In [29]:
# Your code here 
cur.execute(
  """
SELECT *
FROM contactInfo;
"""
).fetchall()

[(1,
  'Christine',
  'Holden',
  'staff',
  2035687697,
  '1672 Whitman Court',
  'Stamford',
  'CT',
  6995),
 (2,
  'Christopher',
  'Warren',
  'student',
  2175150957,
  '1935 University Hill Road',
  'Champaign',
  'IL',
  61938),
 (3,
  'Linda',
  'Jacobson',
  'staff',
  4049446441,
  '479 Musgrave Street',
  'Atlanta',
  'GA',
  30303),
 (4,
  'Andrew',
  'Stepp',
  'student',
  7866419252,
  '2981 Lamberts Branch Road',
  'Hialeah',
  'Fl',
  33012),
 (5,
  'Jane',
  'Evans',
  'student',
  3259909290,
  '1461 Briarhill Lane',
  'Abilene',
  'TX',
  79602),
 (6,
  'Jane',
  'Evans',
  'student',
  3259909290,
  '1461 Briarhill Lane',
  'Abilene',
  'TX',
  79602),
 (7,
  'Mary',
  'Raines',
  'student',
  9075772295,
  '3975 Jerry Toth Drive',
  'Ninilchik',
  'AK',
  99639),
 (8,
  'Ed',
  'Lyman',
  'student',
  5179695576,
  '3478 Be Sreet',
  'Lansing',
  'MI',
  48933)]

## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [30]:
# Your code here
con.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [32]:
# Create the grades table
cur.execute(
"""
CREATE TABLE grades(
                userId INTEGER NOT NULL,
                courseId INTEGER NOT NULL,
                grade TEXT,
                PRIMARY KEY(userId, courseId)

)
"""
)

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [33]:
# Find the duplicate entry
q = """
SELECT *
FROM contactInfo
GROUP BY first_name, last_name, role, telephone_number, street, city, state, zipcode
HAVING COUNT() > 1
"""
pd.read_sql(q,con)

,id,first_name,last_name,role,telephone_number,street,city,state,zipcode
0,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602


In [35]:
# Delete the duplicate entry
cur = con.cursor()
cur.execute("""
DELETE FROM contactInfo
WHERE ROWID NOT IN (
    SELECT MIN(ROWID)
    FROM contactInfo
    GROUP BY first_name, last_name, role, telephone_number, street, city, state, zipcode);
""")
con.commit()

In [36]:
# Check that the duplicate entry was removed
q = """
SELECT *
FROM contactInfo
GROUP BY first_name, last_name, role, telephone_number, street, city, state, zipcode
HAVING COUNT() > 1
"""
pd.read_sql(q,con)

,id,first_name,last_name,role,telephone_number,street,city,state,zipcode


## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [47]:
# Update Ed's address
cur.execute("""
UPDATE contactInfo
SET state = 'PA',
    street = '2910 Simpson Avenue',
    city = 'York',
    zipcode = 17403
WHERE first_name = 'Ed';
""")


In [48]:
# Query the database to ensure the change was made
q = """
SELECT *
FROM contactInfo
WHERE first_name = 'Ed';
"""
pd.read_sql(q,con)

,id,first_name,last_name,role,telephone_number,street,city,state,zipcode
0,8,Ed,Lyman,student,5179695576,2910 Simpson Avenue,York,PA,17403


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [49]:
# Your code here
con.commit()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 